In [1]:
from bayes_opt import BayesianOptimization
from keras.layers import Dense, Dropout, Flatten, Input
from tensorflow import keras
import cv2
import os
import sys
from PIL import Image
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.python.compiler.tensorrt import trt_convert as trt
#from tensorflow.contrib.layers import flatten
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 



from keras.utils import to_categorical

from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events


Using TensorFlow backend.


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
#!pip install bayesian-optimization

# Gathering German Traffic Sign Dataset 

In [8]:
## Downloading Dataset
import urllib.request

print('Beginning file download...')

url = 'https://s3-us-west-1.amazonaws.com/udacity-selfdrivingcar/traffic-signs-data.zip'

urllib.request.urlretrieve(url, './traffic-signs-data.zip')

Beginning file download...


('./traffic-signs-data.zip', <http.client.HTTPMessage at 0x7efd725c5650>)

In [9]:
## Unizping Dataset
import zipfile
import os


print('Beginning file unzip')

zip_ref = zipfile.ZipFile('./traffic-signs-data.zip', 'r')
zip_ref.extractall('./')
zip_ref.close()

print('Done')
os.listdir("./")

Beginning file unzip
Done


['train.p',
 'traffic-signs-data.zip',
 '.ipynb_checkpoints',
 'logs.json',
 '.git',
 '.gitignore',
 'Traffic_Sign_Detection.ipynb',
 'test.p',
 'valid.p']

In [10]:
import pickle
import matplotlib.pyplot as plt
import random
# Visualizations will be shown in the notebook.
%matplotlib inline

In [13]:
training_file = './train.p'
validation_file= './valid.p'
testing_file = './test.p'

In [14]:
with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(validation_file, mode='rb') as f:
    valid = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)

In [15]:
X_train, y_train = train['features'], train['labels']
X_valid, y_valid = valid['features'], valid['labels']
X_test, y_test = test['features'], test['labels']

In [16]:
y_train=to_categorical(y_train)
y_valid=to_categorical(y_valid)
y_test=to_categorical(y_test)

In [17]:
### Replace each question mark with the appropriate value. 
### Use python, pandas or numpy methods rather than hard coding the results

# TODO: Number of training examples
n_train = X_train.shape[0]
# TODO: Number of validation examples
n_validation = X_valid.shape[0]
# TODO: Number of testing examples.
n_test = X_test.shape[0]
# TODO: What's the shape of an traffic sign image?
image_shape = X_train.shape[1:]

# TODO: How many unique classes/labels there are in the dataset.
n_classes = len(np.unique(y_train))

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("Number of validation examples =", n_validation)
print("Image data shape =", image_shape)
print("Number of classes =", n_classes)

Number of training examples = 34799
Number of testing examples = 12630
Number of validation examples = 4410
Image data shape = (32, 32, 3)
Number of classes = 2


In [18]:
# # Visualizations will be shown in the notebook.
# get_ipython().magic('matplotlib inline')

# def draw_images_examples(image_array, grid_x, grid_y, title):
#     fig = plt.figure(figsize=(grid_x,grid_y))
#     fig.suptitle(title, fontsize=20)
 
#     for i in range(1,grid_y*grid_x+1):
#         index = random.randint(0, len(image_array))
#         image = image_array[index].squeeze()
       
#         plt.subplot(grid_y,grid_x,i)
#         plt.imshow(image)
        
# draw_images_examples(X_train, 16, 4, 'Examples of images from training set')
    

# fig = plt.figure(figsize=(12,4))
# n, bins, patches = plt.hist(y_train, n_classes)
# plt.xlabel('Labels')
# plt.ylabel('No. of samples')
# plt.title('Histogram of training samples')

# X_train_one_label = X_train[np.where(y_train==0)]
# draw_images_examples(X_train_one_label, 16, 4, 'Examples of images of the same type - Speed limit (20km/h)')


# CNN Model Training

In [19]:
#import tensorflow as tf
# from keras.layers import Dense, Dropout, Flatten, Input
# from keras.layers import Conv2D, MaxPooling2D
# from keras.layers import BatchNormalization
# from keras.optimizers import Adam
# from keras.models import Sequential

# Model Architecture

In [20]:
hidden_num_units = 2048
hidden_num_units1 = 1024
hidden_num_units2 = 128
output_num_units = 43
pool_size = (2,2)

# Bayesian Optimization

In [21]:
def Black_Box(node0,node1,node2,node3,node4,node5,epochs,Drop0,Drop1,Drop2,Drop3,Drop4,Drop5,batch_size,LR):
    model = keras.Sequential()

    model.add(keras.layers.Conv2D(int(node0), (3, 3), activation='relu', input_shape=(32,32,3), padding='same'))

    model.add(keras.layers.Conv2D(int(node1), (3, 3), activation='relu', padding='same'))
    model.add(keras.layers.MaxPooling2D(pool_size=pool_size))
    model.add(keras.layers.Dropout(int(Drop0)))



    model.add(keras.layers.Conv2D(int(node2), (3, 3), activation='relu', padding='same'))

    model.add(keras.layers.Conv2D(int(node3), (3, 3), activation='relu', padding='same'))
    model.add(keras.layers.MaxPooling2D(pool_size=pool_size))
    model.add(keras.layers.Dropout(int(Drop1)))

    model.add(keras.layers.Conv2D(int(node4), (3, 3), activation='relu', padding='same'))

    model.add(keras.layers.Conv2D(int(node5), (3, 3), activation='relu', padding='same'))
    model.add(keras.layers.MaxPooling2D(pool_size=pool_size))
    model.add(keras.layers.Dropout(int(Drop2)))
    model.add(keras.layers.Flatten())

    model.add(keras.layers.Dense(units=hidden_num_units, activation='relu'))
    model.add(keras.layers.Dropout(int(Drop3)))

    model.add(keras.layers.Dense(units=hidden_num_units1, activation='relu'))
    model.add(keras.layers.Dropout(int(Drop4)))

    model.add(keras.layers.Dense(units=hidden_num_units2, activation='relu'))
    model.add(keras.layers.Dropout(int(Drop5)))

    model.add(keras.layers.Dense(units=output_num_units, input_dim=hidden_num_units, activation='softmax'))
    
    
    model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(lr=LR), metrics=['accuracy'])
    trained_model_conv = model.fit(X_train, y_train, epochs=int(epochs), batch_size=int(batch_size), validation_data=(X_valid, y_valid),verbose=1)
    percentage=model.evaluate (X_train,y_train)[1]*100
    
#     number_of_equal_elements = np.sum(y_train==model.predict(X_train).round())
#     total_elements = np.multiply(y_train.shape)
#     percentage = number_of_equal_elements/total_elements
    return percentage


In [22]:
bounds={'node0':(64,128),
        'node1':(64,128),
        'node2':(32,128),
        'node3':(16,64),
        'node4':(16,64),
        'node5':(16,32),
        'epochs':(10,20),
         'Drop0':(0.1,0.5),
         'Drop1':(0.1,0.5),
         'Drop2':(0.1,0.5),
         'Drop3':(0.1,0.5),
         'Drop4':(0.1,0.5),
         'Drop5':(0.1,0.5),
         'batch_size':(100,400),
         'LR':(0.0005,0.0002)
       }

In [23]:
optimizer = BayesianOptimization(
    f=Black_Box,
    pbounds=bounds,
    random_state=1
)


In [24]:
logger = JSONLogger(path="./logs.json")
optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)

In [ ]:
optimizer.maximize(n_iter=50)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 34799 samples, validate on 4410 samples
Epoch 1/13
34799/34799 [==============================] - 239s 7ms/sample - loss: 1.7249 - acc: 0.5475 - val_loss: 1.1175 - val_acc: 0.7274
Epoch 2/13
34799/34799 [==============================] - 240s 7ms/sample - loss: 0.3080 - acc: 0.9145 - val_loss: 0.8614 - val_acc: 0.8463
Epoch 3/13
34799/34799 [==============================] - 229s 7ms/sample - loss: 0.1389 - acc: 0.9629 - val_loss: 0.7989 - val_acc: 0.8714
Epoch 4/13
34799/34799 [==============================] - 240s 7ms/sample - loss: 0.0714 - acc: 0.9809 - val_loss: 0.6480 - val_acc: 0.8823
Epoch 5/13
34799/34799 [==============================] - 250s 7ms/sample - loss: 0.0642 - acc: 0.9829 - val_loss: 0.6683 - val_acc: 0.8927
Epoch 6/13
34799/34799 [==============================] - 245s 7ms/sample - loss: 0.0457 - acc: 0.9879 - val_loss: 0.8935 - val_acc: 0.8982
Epoch 7/13
34799/34799 [======

# No bayesian optimization

In [ ]:
epochs = 15
batch_size = 200
pool_size = (2,2)
#list_images /= 255.0
input_shape = Input(shape=(32, 32,3))

In [ ]:
model = keras.Sequential()

model.add(keras.layers.Conv2D(128, (3, 3), activation='relu', input_shape=(32,32,3), padding='same'))

model.add(keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(keras.layers.MaxPooling2D(pool_size=pool_size))
model.add(keras.layers.Dropout(0.2))
          

          
model.add(keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'))

model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(keras.layers.MaxPooling2D(pool_size=pool_size))
model.add(keras.layers.Dropout(0.2))
    
model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same'))

model.add(keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same'))
model.add(keras.layers.MaxPooling2D(pool_size=pool_size))
model.add(keras.layers.Dropout(0.2))
model.add(keras.layers.Flatten())
          
model.add(keras.layers.Dense(units=hidden_num_units, activation='relu'))
model.add(keras.layers.Dropout(0.2))
          
model.add(keras.layers.Dense(units=hidden_num_units1, activation='relu'))
model.add(keras.layers.Dropout(0.3))
          
model.add(keras.layers.Dense(units=hidden_num_units2, activation='relu'))
model.add(keras.layers.Dropout(0.3))
          
model.add(keras.layers.Dense(units=output_num_units, input_dim=hidden_num_units, activation='softmax'))


In [1]:
#model.compile(loss=tf.keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(lr=1e-4), metrics=['accuracy'])
#trained_model_conv = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_valid, y_valid),verbose=1)

# Plot of the learning history of the trained CNN model 

In [ ]:
plt.figure(1)
# summarize history for accuracy
plt.subplot(211)
plt.plot(trained_model_conv.history['acc'])
#plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
# summarize history for loss
plt.subplot(212)
plt.plot(trained_model_conv.history['loss'])
#plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:
score=model.evaluate(X_test, y_test)
print('loss:', score[0])
print('accuracy:', score[1])


In [ ]:

# #import tensorflow as tf
# #from tensorflow.contrib.layers import flatten
# from tqdm import tqdm

# def grayscale(img):    
#     return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)[:,:,None]

# def normalize(value):
#     return value / 255 * 2 - 1

# def preprocess_image(image):
#     img = grayscale(image)
#     img = normalize(img)
#     return img


# def preprocess_batch(images):
#     imgs = np.zeros(shape=images.shape)
#     processed_image_depth = preprocess_image(images[0]).shape[2]
#     imgs = imgs[:,:,:,0:processed_image_depth]
#     for i in tqdm(range(images.shape[0])):
#         imgs[i] = preprocess_image(images[i])        
#     return imgs

# X_train_processed = preprocess_batch(X_train)
# X_valid_processed = preprocess_batch(X_valid)
# X_test_processed = preprocess_batch(X_test)

# no_test_image = 10000

# sample_image = X_train[no_test_image]
# sample_image_processed = grayscale(X_train[no_test_image])
# fig=plt.figure(figsize=(16,3))
# sub=plt.subplot(131)
# sub.set_title("Original image")
# plt.imshow(sample_image)
# sub=plt.subplot(132)
# sub.set_title("Preprocessed image")
# plt.imshow(sample_image_processed.squeeze(), cmap='gray')

# print("Sample image dimension BEFORE processing: {}".format(sample_image.shape))
# print("Sample image dimension AFTER processing: {}".format(sample_image_processed.shape))

# image_depth = X_train_processed.shape[3]

# sample_image = X_train[no_test_image]
# dim1 = sample_image.shape[0]
# dim2 = sample_image.shape[1]
# dim3 = sample_image.shape[2]
# sample_image_reshaped = np.reshape(sample_image, dim1*dim2*dim3)
# plt.figure(figsize=(16,3))
# sub=plt.subplot(131)
# sub.set_title("Original image histogram")
# n, bins, patches = plt.hist(sample_image_reshaped, 255)

# sample_image_processed = X_train_processed[no_test_image]
# dim1 = sample_image_processed.shape[0]
# dim2 = sample_image_processed.shape[1]
# dim3 = sample_image_processed.shape[2]
# sample_image_processed_reshaped = np.reshape(sample_image_processed, dim1*dim2*dim3)
# sub=plt.subplot(132)
# sub.set_title("Preprocessed image histogram")
# n, bins, patches = plt.hist(sample_image_processed_reshaped,255)

# Useful Articles
- https://github.com/tomaszkacmajor/CarND-Traffic-Sign-Classifier-P2/blob/master/Traffic_Sign_Classifier.ipynb
- https://medium.com/typeiqs/traffic-sign-recognition-aa38d699ac9 
- https://medium.com/@sdoshi579/convolutional-neural-network-learn-and-apply-3dac9acfe2b6
- https://towardsdatascience.com/traffic-sign-detection-using-convolutional-neural-network-660fb32fe90e    
    